# Cabinet Start-Up Procedure

1. Turn on cabinet power in back of cabinet
2. Turn on sensor power (24 Volts). The sensor power inputs are labeled.
3. Compile code to AMDC
4. Turn on power to DC bus of cabinet.
5. Run Jupyter Notebook. 

Import neccessary modules

In [ ]:
amdc.disconnect()



In [ ]:
from Cabinet_Test import cmd
import sys
sys.path.append(r'..\AMDC-Firmware\scripts')
from AMDC import AMDC
amdc = AMDC(port = 'COM13', cmdEcho = True, printOutput=True)
Vdc = 30
cmd = cmd(Vdc, amdc, debug = False) 

In [ ]:
  
cmd.disable_pwm()
cmd.BIM_init()
cmd.BIM_set_Vdc(30)
cmd.enable_pwm()


In [ ]:
cmd.BIM_enable_ctrl()
cmd.BIM_enable_ob()

In [65]:
import matplotlib
import time
import pathlib as pl
import sys
repo_dir = '..'                                      # CHANGE THIS TO YOUR REPO DIRECTORY
repo_dir = pl.Path(repo_dir)

from AMDC_Logger import AMDC_Logger, find_mapfile
mapfile_path = find_mapfile(repo_dir)
logger = AMDC_Logger(AMDC = amdc, mapfile = mapfile_path)
logger.sync()

# View which variables are logged
logger.info()
#amdc.connect()
cmd.twin_enable_log()

	> log info
	> log info
###################   AMDC LOGGER INFO   ###################

Variable Name:      Index:         Type:         Sample Rate [Hz]:    Number of Samples:

	> twin enable_log
['twin enable_log', 'SUCCESS']


In [ ]:
logger.unregister_all()
#logger.register('Ia1_a Ia1_b Ia1_c Ib1_a Ib1_b Ib1_c Ia2_a Ia2_b Ia2_c Ib2_a Ib2_b Ib2_c', samples_per_sec = 1000, var_type = 'double')
#logger.register('theta_rm wrm wsl theta_e we Itq_d_ref Itq_q_ref Itq_d Itq_q Te', samples_per_sec = 1000, var_type = 'double')
#logger.register('theta_rm theta_e wsl', samples_per_sec = 1000, var_type = 'double')
logger.register('Itq_d Itq_d_ref', samples_per_sec = 1000, var_type = 'double')



# View which variables are logged
logger.info()


In [ ]:

###################   COLLECT DATA   ####################
# Clear the logger, then record data
logger.clear_all()
logger.start()



In [ ]:
cmd.BIM_enable_vctrl()
cmd.BIM_set_id(2)

#cmd.BIM_enable_levctrl()
#cmd.enable_pwm()
#cmd.twin_enable_log()

In [ ]:

#DATA IS BEING RECORDED
#time.sleep(1)

logger.stop()

####################   DUMP DATA AND PLOT   ####################
data = logger.dump(file = 'BIM_data_pos.csv')

In [ ]:

data.filter(regex="^wrm$").plot()
#data.filter(regex="^we$").plot()
data.filter(regex="^wsl$").plot()
data.filter(regex="^theta_e|theta_rm$").plot()
#data.filter(regex="^theta_rm$").plot()
data.filter(regex="^Itq_d_ref|Itq_d$").plot()
data.filter(regex="^Itq_q_ref|Itq_q$").plot()
data.filter(regex="^Te$").plot()

In [ ]:
logger.unregister_all()
logger.clear_all()


In [ ]:
for i in [150, 200]:
   
    logger.stop()
    logger.unregister_all()
    logger.clear_all()
    sampling = 10000
    period = 1
    if i <= 10:
        sampling = 10000
        period = 1
    elif i<= 100:
        sampling = 10000
        period = 0.2

    else:
        sampling = 10000
        period = 0.1

    logger.unregister_all()
    #logger.register('Ia1_a Ia1_b Ia1_c Ib1_a Ib1_b Ib1_c Ia2_a Ia2_b Ia2_c Ib2_a Ib2_b Ib2_c', samples_per_sec = 1000, var_type = 'double')
    #logger.register('Itq_d Itq_q vd_ref vq_ref Is1_x Is1_y vx_ref vy_ref', samples_per_sec = 1000, var_type = 'double')
    logger.register('Itq_d vd_ref', samples_per_sec = sampling, var_type = 'double')

    ###################   COLLECT DATA   ####################
    # Clear the logger, then record data
   
    logger.clear_all()
    vd_ref_chirp_gain     = 500 * 1e-3; # [m]
    vd_ref_chirp_freq_min = i; # [Hz]
    vd_ref_chirp_freq_max = i; # [Hz]
    vd_ref_chirp_period   = period; # [sec]
    amdc.cmd("inj const vq_ref set 0")
    amdc.cmd("inj const vx_ref set 0")
    amdc.cmd("inj const vy_ref set 0")
    amdc.cmd("inj chirp vd_ref add {:.6f} {:.6f} {:.6f} {:.6f}".format(vd_ref_chirp_gain, vd_ref_chirp_freq_min, vd_ref_chirp_freq_max, vd_ref_chirp_period))

    logger.start()
    #DATA IS BEING RECORDED
    time.sleep(period*1.1)

    logger.stop()

    ####################   DUMP DATA AND PLOT   ####################
    filename = 'BIM_data_id_vd'+ str(i)+'.csv'
    data = logger.dump(file = filename)

    logger.unregister_all()
    #logger.register('Ia1_a Ia1_b Ia1_c Ib1_a Ib1_b Ib1_c Ia2_a Ia2_b Ia2_c Ib2_a Ib2_b Ib2_c', samples_per_sec = 1000, var_type = 'double')
    #logger.register('Itq_d Itq_q vd_ref vq_ref Is1_x Is1_y vx_ref vy_ref', samples_per_sec = 1000, var_type = 'double')
    logger.register('Is1_x vx_ref', samples_per_sec = sampling, var_type = 'double')

    ###################   COLLECT DATA   ####################
    # Clear the logger, then record data

    logger.clear_all()
    amdc.cmd("inj const vd_ref set 0")
    amdc.cmd("inj const vq_ref set 0")
    amdc.cmd("inj const vy_ref set 0")
    amdc.cmd("inj chirp vx_ref add {:.6f} {:.6f} {:.6f} {:.6f}".format(vd_ref_chirp_gain, vd_ref_chirp_freq_min, vd_ref_chirp_freq_max, vd_ref_chirp_period))

    logger.start()
    #DATA IS BEING RECORDED
    time.sleep(period*1.1)

    logger.stop()

    ####################   DUMP DATA AND PLOT   ####################
    filename = 'BIM_data_id_vx'+ str(i)+'.csv'
    data = logger.dump(file = filename)

In [70]:
for i in [50, 1200, 2000]:
   
    logger.stop()
    logger.unregister_all()
    logger.clear_all()
    sampling = 1000
    period = 1
    if i <= 10:
        sampling = 1000
        period = 1
    elif i<= 100:
        sampling = 1000
        period = 0.2

    else:
        sampling = 10000
        period = 0.1

    logger.unregister_all()
    #logger.register('Ia1_a Ia1_b Ia1_c Ib1_a Ib1_b Ib1_c Ia2_a Ia2_b Ia2_c Ib2_a Ib2_b Ib2_c', samples_per_sec = 1000, var_type = 'double')
    #logger.register('Itq_d Itq_q vd_ref vq_ref Is1_x Is1_y vx_ref vy_ref', samples_per_sec = 1000, var_type = 'double')
    logger.register('Itq_d Itq_d_ref Itq_q Is1_y Is1_x', samples_per_sec = sampling, var_type = 'double')

    ###################   COLLECT DATA   ####################
    # Clear the logger, then record data

    logger.clear_all()
    cmd.BIM_set_id(2)
    vd_ref_chirp_gain     = 500 * 1e-3; # [m]
    vd_ref_chirp_freq_min = i; # [Hz]
    vd_ref_chirp_freq_max = i; # [Hz]
    vd_ref_chirp_period   = period; # [sec]
    amdc.cmd("inj const ix_ref set 0")
    amdc.cmd("inj const iy_ref set 0")
    amdc.cmd("inj const iq_ref set 0")
    amdc.cmd("inj chirp id_ref set {:.6f} {:.6f} {:.6f} {:.6f}".format(vd_ref_chirp_gain, vd_ref_chirp_freq_min, vd_ref_chirp_freq_max, vd_ref_chirp_period))

    logger.start()
    #DATA IS BEING RECORDED
    time.sleep(period*1.1)

    logger.stop()

    ####################   DUMP DATA AND PLOT   ####################
    filename = 'BIM_data_id_id'+ str(i)+'.csv'
    data = logger.dump(file = filename)

    logger.unregister_all()
    #logger.register('Ia1_a Ia1_b Ia1_c Ib1_a Ib1_b Ib1_c Ia2_a Ia2_b Ia2_c Ib2_a Ib2_b Ib2_c', samples_per_sec = 1000, var_type = 'double')
    #logger.register('Itq_d Itq_q vd_ref vq_ref Is1_x Is1_y vx_ref vy_ref', samples_per_sec = 1000, var_type = 'double')
    logger.register('Itq_d Is1_x_ref Itq_q Is1_y Is1_x', samples_per_sec = sampling, var_type = 'double')

    ###################   COLLECT DATA   ####################
    # Clear the logger, then record data
    logger.clear_all()
    cmd.BIM_set_ixy(0,0)
    amdc.cmd("inj const id_ref set 0")
    amdc.cmd("inj const iq_ref set 0")
    amdc.cmd("inj const iy_ref set 0")
    amdc.cmd("inj chirp ix_ref add {:.6f} {:.6f} {:.6f} {:.6f}".format(vd_ref_chirp_gain, vd_ref_chirp_freq_min, vd_ref_chirp_freq_max, vd_ref_chirp_period))

    logger.start()
    #DATA IS BEING RECORDED
    time.sleep(period*1.1)

    logger.stop()

    ####################   DUMP DATA AND PLOT   ####################
    filename = 'BIM_data_id_ix'+ str(i)+'.csv'
    data = logger.dump(file = filename)

	> log stop
['log stop', 'FAILURE']
	> log unreg 0
['log unreg 0', 'SUCCESS']
	> log unreg 1
['log unreg 1', 'SUCCESS']
	> log unreg 2
['log unreg 2', 'SUCCESS']
	> log unreg 3
['log unreg 3', 'SUCCESS']
	> log unreg 4
['log unreg 4', 'SUCCESS']
	> log empty_all
['log empty_all', 'SUCCESS']
	> log reg 0 LOG_Itq_d 1230432 1000 double
['log reg 0 LOG_Itq_d 1230432 1000 double', 'SUCCESS']
	> log reg 1 LOG_Itq_d_ref 1230400 1000 double
['log reg 1 LOG_Itq_d_ref 1230400 1000 double', 'SUCCESS']
	> log reg 2 LOG_Itq_q 1230440 1000 double
['log reg 2 LOG_Itq_q 1230440 1000 double', 'SUCCESS']
	> log reg 3 LOG_Is1_y 1230456 1000 double
['log reg 3 LOG_Is1_y 1230456 1000 double', 'SUCCESS']
	> log reg 4 LOG_Is1_x 1230448 1000 double
['log reg 4 LOG_Is1_x 1230448 1000 double', 'SUCCESS']
	> log empty_all
['log empty_all', 'SUCCESS']
	> BIM set_id 2.000000
['BIM set_id 2.000000', 'SUCCESS']
	> inj const ix_ref set 0
['inj const ix_ref set 0', 'SUCCESS']
	> inj const iy_ref set 0
['inj const iy_r

In [ ]:

logger.unregister_all()
#logger.register('Ia1_a Ia1_b Ia1_c Ib1_a Ib1_b Ib1_c Ia2_a Ia2_b Ia2_c Ib2_a Ib2_b Ib2_c', samples_per_sec = 1000, var_type = 'double')
#logger.register('Itq_d Itq_q vd_ref vq_ref Is1_x Is1_y vx_ref vy_ref', samples_per_sec = 1000, var_type = 'double')
logger.register('Itq_d vd_ref', samples_per_sec = 10000, var_type = 'double')


In [ ]:

###################   COLLECT DATA   ####################
# Clear the logger, then record data
d_ref_chirp_gain     = 1000 * 1e-3; # [m]
vd_ref_chirp_freq_min = 150; # [Hz]
vd_ref_chirp_freq_max = 150; # [Hz]
vd_ref_chirp_period   = 0.3; # [sec]
logger.clear_all()

amdc.cmd("inj const vq_ref set 0")
amdc.cmd("inj const vx_ref set 0")
amdc.cmd("inj const vy_ref set 0")
amdc.cmd("inj chirp vd_ref set {:.6f} {:.6f} {:.6f} {:.6f}".format(vd_ref_chirp_gain, vd_ref_chirp_freq_min, vd_ref_chirp_freq_max, vd_ref_chirp_period))
logger.start()

#DATA IS BEING RECORDED
time.sleep(0.1)

logger.stop()

####################   DUMP DATA AND PLOT   ####################
data = logger.dump(file = 'BIM_data_id_vd150.csv')

logger.unregister_all()
#logger.register('Ia1_a Ia1_b Ia1_c Ib1_a Ib1_b Ib1_c Ia2_a Ia2_b Ia2_c Ib2_a Ib2_b Ib2_c', samples_per_sec = 1000, var_type = 'double')
#logger.register('Itq_d Itq_q vd_ref vq_ref Is1_x Is1_y vx_ref vy_ref', samples_per_sec = 1000, var_type = 'double')
logger.register('Is1_x vx_ref', samples_per_sec = 20000, var_type = 'double')

###################   COLLECT DATA   ####################
# Clear the logger, then record data
logger.clear_all()

amdc.cmd("inj const vd_ref set 0")
amdc.cmd("inj const vq_ref set 0")
amdc.cmd("inj const vy_ref set 0")
amdc.cmd("inj chirp vx_ref set {:.6f} {:.6f} {:.6f} {:.6f}".format(vd_ref_chirp_gain, vd_ref_chirp_freq_min, vd_ref_chirp_freq_max, vd_ref_chirp_period))
logger.start()

#DATA IS BEING RECORDED
time.sleep(0.1)

logger.stop()

####################   DUMP DATA AND PLOT   ####################
data = logger.dump(file = 'BIM_data_id_vx150.csv')


In [ ]:

logger.unregister_all()
#logger.register('Ia1_a Ia1_b Ia1_c Ib1_a Ib1_b Ib1_c Ia2_a Ia2_b Ia2_c Ib2_a Ib2_b Ib2_c', samples_per_sec = 1000, var_type = 'double')
#logger.register('Itq_d Itq_q vd_ref vq_ref Is1_x Is1_y vx_ref vy_ref', samples_per_sec = 1000, var_type = 'double')
logger.register('Is1_y vy_ref', samples_per_sec = 10000, var_type = 'double')

###################   COLLECT DATA   ####################
# Clear the logger, then record data
logger.clear_all()
logger.start()
amdc.cmd("inj const vd_ref set 0")
amdc.cmd("inj const vq_ref set 0")
amdc.cmd("inj const vx_ref set 0")
amdc.cmd("inj chirp vy_ref set {:.6f} {:.6f} {:.6f} {:.6f}".format(vd_ref_chirp_gain, vd_ref_chirp_freq_min, vd_ref_chirp_freq_max, vd_ref_chirp_period))


#DATA IS BEING RECORDED
time.sleep(0.5)

logger.stop()

####################   DUMP DATA AND PLOT   ####################
data = logger.dump(file = 'BIM_data_id_vy200.csv')

In [ ]:
amdc.cmd("inj const vd_ref set 0")

In [ ]:

#DATA IS BEING RECORDED
time.sleep(4)

logger.stop()


In [ ]:
####################   DUMP DATA AND PLOT   ####################
data = logger.dump(file = 'BIM_data_id.csv')


In [ ]:
####################   DUMP DATA AND PLOT   ####################

#data.filter(regex="^theta_rm$").plot()
data.filter(regex="^vd_ref|Itq_d$").plot()
#data.filter(regex="^vq_ref|Itq_q$").plot()
#data.filter(regex="^vx_ref|Is1_x$").plot()
#data.filter(regex="^vy_ref|Is1_y$").plot()


In [ ]:
cmd.BIM_enable_ob()

In [ ]:
cmd.BIM_set_Te(0.2)

In [ ]:
cmd.BIM_set_w(10)

In [ ]:
cmd.BIM_set_ixy(0, 0)
cmd.BIM_set_id(0)

In [71]:
cmd.disable_pwm()

	> hw pwm off
['hw pwm off', 'SUCCESS']


In [ ]:
vd_ref_chirp_gain     = 500 * 1e-3; # [m]
vd_ref_chirp_freq_min = 1; # [Hz]
vd_ref_chirp_freq_max = 100; # [Hz]
vd_ref_chirp_period   = 1.0; # [sec]
amdc.cmd("inj chirp vd_ref set {:.6f} {:.6f} {:.6f} {:.6f}".format(vd_ref_chirp_gain, vd_ref_chirp_freq_min, vd_ref_chirp_freq_max, vd_ref_chirp_period))

In [ ]:
vd_ref_chirp_gain     = 500 * 1e-3; # [m]
vd_ref_chirp_freq_min = 1; # [Hz]
vd_ref_chirp_freq_max = 100; # [Hz]
vd_ref_chirp_period   = 1.0; # [sec]
amdc.cmd("inj chirp vd_ref set {:.6f} {:.6f} {:.6f} {:.6f}".format(vd_ref_chirp_gain, vd_ref_chirp_freq_min, vd_ref_chirp_freq_max, vd_ref_chirp_period))

In [ ]:
amdc.disconnect()
#amdc.connect()

## Logging


In [ ]:
import matplotlib
import time
import pathlib as pl
import sys
repo_dir = '..'                                      # CHANGE THIS TO YOUR REPO DIRECTORY
repo_dir = pl.Path(repo_dir)

from AMDC_Logger import AMDC_Logger, find_mapfile
mapfile_path = find_mapfile(repo_dir)
logger = AMDC_Logger(AMDC = amdc, mapfile = mapfile_path)
logger.sync()

# View which variables are logged
logger.info()
#amdc.connect()
cmd.twin_enable_log()

In [ ]:
logger.unregister_all()
#logger.register('Ia1_a Ia1_b Ia1_c Ib1_a Ib1_b Ib1_c Ia2_a Ia2_b Ia2_c Ib2_a Ib2_b Ib2_c', samples_per_sec = 1000, var_type = 'double')
#logger.register('theta_rm wrm wsl theta_e we Itq_d_ref Itq_q_ref Itq_d Itq_q Te', samples_per_sec = 1000, var_type = 'double')
logger.register('theta_rm theta_e wrm', samples_per_sec = 1000, var_type = 'double')



# View which variables are logged
logger.info()

###################   COLLECT DATA   ####################
# Clear the logger, then record data
logger.clear_all()
logger.start()


In [ ]:

#DATA IS BEING RECORDED
time.sleep(1)

logger.stop()


In [ ]:
####################   DUMP DATA AND PLOT   ####################
data = logger.dump(file = 'BIM_data_pos.csv')


In [ ]:
data.filter(regex="^wrm$").plot()
#data.filter(regex="^we$").plot()
#data.filter(regex="^wsl$").plot()
data.filter(regex="^theta_e|theta_rm$").plot()
#data.filter(regex="^theta_rm$").plot()
#data.filter(regex="^Itq_d_ref|Itq_d$").plot()
#data.filter(regex="^Itq_q_ref|Itq_q$").plot()
#data.filter(regex="^Te$").plot()


In [ ]:

####################   DUMP DATA AND PLOT   ####################
data = logger.dump(file = 'BIM_data_pos.csv')
#data


In [ ]:
data.filter(regex="^wrm$").plot()
#data.filter(regex="^we$").plot()
data.filter(regex="^wsl$").plot()
data.filter(regex="^theta_e|theta_rm$").plot()
#data.filter(regex="^theta_rm$").plot()
data.filter(regex="^Itq_d_ref|Itq_d$").plot()
data.filter(regex="^Itq_q_ref|Itq_q$").plot()
data.filter(regex="^Te$").plot()


In [ ]:
logger.unregister_all()
logger.register('Iabc1_a Iabc1_b Iabc1_c', samples_per_sec = 1000, var_type = 'double')

# View which variables are logged
logger.info()

logger.clear_all()
logger.start()

# DATA IS BEING RECORDED
#time.sleep(1)

logger.stop()

data = logger.dump(file = 'test_data_Iabc.csv')
#data

data.filter(regex="^Iabc1_a|Iabc1_b|Iabc1_c$").plot()

In [ ]:
logger.unregister_all()
#logger.register('Itq_d_ref Itq_q_ref Itq_d Itq_q Is1_d_ref Is1_q_ref Is1_d Is1_q Is2_d_ref Is2_q_ref Is2_d Is2_q', samples_per_sec = 1000, var_type = 'double')
logger.register('va_ref vb_ref vc_ref', samples_per_sec = 1000, var_type = 'double')
# View which variables are logged
logger.info()

logger.clear_all()
logger.start()
time.sleep(0.5)
logger.stop()

data = logger.dump(file = 'BIM_data_Idq.csv')
#data

data.filter(regex="^va_ref|vb_ref|vc_ref$").plot()
#data.filter(regex="^Itq_q_ref|Itq_q$").plot()
#data.filter(regex="^Te_ref$").plot()

In [ ]:
data.filter(regex="^Ib1_a|Ib1_b|Ib1_c$").plot()

In [ ]:
data.filter(regex="^Iabc2_a|Iabc2_b|Iabc2_c$").plot()

In [ ]:
data.filter(regex="^Iabc3_a|Iabc3_b|Iabc3_c$").plot()

In [ ]:
logger.unregister_all()
logger.register('vabc1_a vabc1_b vabc1_c', samples_per_sec = 1000, var_type = 'double')

# View which variables are logged
logger.info()

logger.clear_all()
logger.start()

# DATA IS BEING RECORDED
time.sleep(1)

logger.stop()

data = logger.dump(file = 'test_data.csv')
data

data.filter(regex="^vabc1_a|vabc1_b|vabc1_c$").plot()

In [ ]:

data = logger.dump(file = 'BIM_data.csv')
data


In [ ]:
data.filter(regex="^Itq_d_ref|Itq_q_ref|Itq_d$|Itq_q").plot()
#data.filter(regex="^Is1_d_ref|Is1_q_ref|Is1_d|Is1_q$").plot()
#data.filter(regex="^Is2_d_ref|Is2_q_ref|Is2_d|Is2_q$").plot()

In [ ]:
logger.unregister_all()
logger.register('Is1_d_ref Is1_q_ref Is1_d Is1_q', samples_per_sec = 1000, var_type = 'double')

# View which variables are logged
logger.info()

logger.clear_all()
logger.start()

# DATA IS BEING RECORDED
time.sleep(1)

logger.stop()


In [ ]:
#data = logger.dump(file = 'test_data.csv')
#data
data.filter(regex="^Is1_d_ref|Is1_q_ref|Is1_d|Is1_q$").plot()

In [ ]:
logger.unregister_all()
logger.register('Is2_d_ref Is2_q_ref Is2_d Is2_q', samples_per_sec = 1000, var_type = 'double')

# View which variables are logged
logger.info()

logger.clear_all()
logger.start()

# DATA IS BEING RECORDED
time.sleep(1)

logger.stop()

data = logger.dump(file = 'test_data.csv')
data

data.filter(regex="^Is2_d_ref|Is2_q_ref|Is2_d|Is2_q$").plot()

In [ ]:

logger.unregister_all()
logger.register('v_tq_d v_tq_q', samples_per_sec = 1000, var_type = 'double')

# View which variables are logged
logger.info()

logger.clear_all()
logger.start()

# DATA IS BEING RECORDED
time.sleep(1)

logger.stop()

data = logger.dump(file = 'test_data.csv')
data

data.filter(regex="^v_tq_d|v_tq_q$").plot()

In [ ]:
logger.unregister_all()
logger.register('we_tq', samples_per_sec = 1000, var_type = 'double')

# View which variables are logged
logger.info()

logger.clear_all()
logger.start()

# DATA IS BEING RECORDED
time.sleep(1)

logger.stop()

data = logger.dump(file = 'test_data.csv')
data

data.filter(regex="^we_tq$").plot()